<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.4: Functional Programming
**Prev: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**<br>
**Next: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**

## Motivation
You saw functions in many previous modules, but now it's time to make our own and use them effectively.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc #2

path: String = "/Users/dan/dev/dan-fritchman/chisel-bootcamp/source/load-ivy.sc"

This module uses the Chisel `FixedPoint` type, which currently resides in the experimental package.

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.experimental._
import chisel3.internal.firrtl.KnownBinaryPoint

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3.experimental._

import chisel3.internal.firrtl.KnownBinaryPoint

---
# Functional Programming in Scala
Scala functions were introduced in Module 1, and you saw then used a lot in the previous module. Here's a refresher on functions. Functions take any number of inputs and produce one output. Inputs are often called arguments to a function. To produce no output, return the `Unit` type. 

<span style="color:blue">**Example: Custom Functions**</span><br>
Below are some examples of functions in Scala.

In [3]:
// No inputs or outputs (two versions).
def hello1(): Unit = print("Hello!")
def hello2 = print("Hello again!")

// Math operation: one input and one output.
def times2(x: Int): Int = 2 * x

// Inputs can have default values, and explicitly specifying the return type is optional.
// Note that we recommend specifying the return types to avoid surprises/bugs.
def timesN(x: Int, n: Int = 2) = n * x

// Call the functions listed above.
hello1()
hello2
times2(4)
timesN(4)         // no need to specify n to use the default value
timesN(4, 3)      // argument order is the same as the order where the function was defined
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly

Hello!Hello again!

defined function hello1
defined function hello2
defined function times2
defined function timesN
res2_6: Int = 8
res2_7: Int = 8
res2_8: Int = 12
res2_9: Int = 14

## Functions as Objects
Functions in Scala are first-class objects. That means we can assign a function to a `val` and pass it to classes, objects, or other functions as an argument.

<span style="color:blue">**Example: Function Objects**</span><br>
Below are the same functions implemented as functions and as objects.

In [4]:
// These are normal functions.
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// These are functions as vals.
// The first one explicitly specifies the return type.
val plus1val: Int => Int = x => x + 1
val times2val = (x: Int) => x * 2

// Calling both looks the same.
plus1funct(4)
plus1val(4)
plus1funct(x=4)
//plus1val(x=4) // this doesn't work

defined function plus1funct
defined function times2funct
plus1val: Int => Int = ammonite.$sess.cmd3$Helper$$Lambda$3503/0x0000000802070840@e09f12e
times2val: Int => Int = ammonite.$sess.cmd3$Helper$$Lambda$3504/0x0000000802071040@50820e5e
res3_4: Int = 5
res3_5: Int = 5
res3_6: Int = 5

Why would you want to create a `val` instead of a `def`? With a `val`, you can now pass the function around to other functions, as shown below. You can even create your own functions that accept other functions as arguments. Formally, functions that take or produce functions are called *higher-order functions*. You saw them used in the last module, but now you'll make your own!

<span style="color:blue">**Example: Higher-Order Functions**</span><br>
Here we show `map` again, and we also create a new function, `opN`, that accepts a function, `op`, as an argument.

In [5]:
// create our function
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)
val myListTimes = myList.map(times2)

// create a custom function, which performs an operation on X N times using recursion
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)
opN(7, 3, times2)

plus1: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$3529/0x0000000802087840@70bfbd15
times2: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$3530/0x0000000802088840@1f369155
myList: List[Int] = List(1, 2, 5, 9)
myListPlus: List[Int] = List(2, 3, 6, 10)
myListTimes: List[Int] = List(2, 4, 10, 18)
defined function opN
res4_6: Int = 10
res4_7: Int = 56

<span style="color:blue">**Example: Functions vs. Objects**</span><br>
A possibly confusing situation arises when using functions without arguments. Functions are evaluated every time they are called, while `val`s are evaluated at instantiation. 

In [6]:
import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function
val x = Random.nextInt
def y = Random.nextInt

// x was previously evaluated, so it is a constant
println(s"x = $x")
println(s"x = $x")

// y is a function and gets reevaluated at each call, thus these produce different results
println(s"y = $y")
println(s"y = $y")

x = -542328862
x = -542328862
y = -1644996740
y = -1236732432


import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function

x: Int = -542328862
defined function y

## Anonymous Functions
As the name implies, anonymous functions are nameless. There's no need to create a `val` for a function if we'll only use it once. 

<span style="color:blue">**Example: Anonymous Functions**</span><br>
The following example demonstrates this. They are often scoped (put in curly braces instead of parentheses). 

In [7]:
val myList = List(5, 6, 7, 8)

// add one to every item in the list using an anonymous function
// arguments get passed to the underscore variable
// these all do the same thing
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

// a common situation is to use case statements within an anonymous function
val myAnyList = List(1, 2, "3", 4L, myList)
myAnyList.map {
  case (_:Int|_:Long) => "Number"
  case _:String => "String"
  case _ => "error"
}

myList: List[Int] = List(5, 6, 7, 8)
res6_1: List[Int] = List(6, 7, 8, 9)
res6_2: List[Int] = List(6, 7, 8, 9)
myAnyList: List[Any] = List(1, 2, "3", 4L, List(5, 6, 7, 8))
res6_4: List[String] = List("Number", "Number", "String", "Number", "error")

<span style="color:red">**Exercise: Sequence Manipulation**</span><br>
A common set of higher-order functions you'll use are `scanLeft`/`scanRight`, `reduceLeft`/`reduceRight`, and `foldLeft`/`foldRight`. It's important to understand how each one works and when to use them. The default directions for `scan`, `reduce`, and `fold` are left, though this is not guaranteed for all cases. 

In [16]:
val exList = List(1, 5, 7, 100)

// write a custom function to add two numbers, then use reduce to find the sum of all values in exList
def add(a: Int, b: Int): Int = a+b
val sum = exList.reduce(add)

// find the sum of exList using an anonymous function (hint: you've seen this before!)
val anon_sum = exList.reduce(_+_)

// find the moving average of exList from right to left using scan; make the result (ma2) a list of doubles
def avg(a: Int, b: Double): Double = (a+b)/2.0
val ma2 = exList.scanRight(0.0)(avg)

exList: List[Int] = List(1, 5, 7, 100)
defined function add
sum: Int = 113
anon_sum: Int = 113
defined function avg
ma2: List[Double] = List(8.875, 16.75, 28.5, 50.0, 0.0)

In [17]:
assert(add(88, 88) == 176)
assert(sum == 113)

assert(anon_sum == 113)

assert(avg(100, 100.0) == 100.0)
assert(ma2 == List(8.875, 16.75, 28.5, 50.0, 0.0))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

val anon\_sum = exList.reduce(\_ + \_)

def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)
</pre></article></div></section></div>

---
# Functional Programming in Chisel
Let's look at some examples of how to use functional programming when creating hardware generators in Chisel.

<span style="color:blue">**Example: FIR Filter**</span><br>
First, we'll revisit the FIR filter from previous examples. Instead of passing in the coefficients as parameters to the class or making them programmable, we'll pass a function to the FIR that defines how the window coefficients are calculated. This function will take the window length and bitwidth to produce a scaled list of coefficients. Here are two example windows. To avoid fractions, we'll scale the coefficients to be between the max and min integer values. For more on these windows, check out the [this Wikipedia page](https://en.wikipedia.org/wiki/Window_function).

In [19]:
// get some math functions
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window
val TriangularWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window
val HammingWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// check it out! first argument is the window length, and second argument is the bitwidth
TriangularWindow(10, 16)
HammingWindow(10, 16)

import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window

TriangularWindow: (Int, Int) => Seq[Int] = ammonite.$sess.cmd18$Helper$$Lambda$4216/0x0000000801ebe840@501cae04
HammingWindow: (Int, Int) => Seq[Int] = ammonite.$sess.cmd18$Helper$$Lambda$4217/0x0000000801ebd040@70646551
res18_3: Seq[Int] = Vector(
  0,
  14564,
  29127,
  43691,
  58254,
  58254,
  43691,
  29127,
  14564,
  0
)
res18_4: Seq[Int] = Vector(
  5243,
  12296,
  30155,
  50463,
  63718,
  63718,
  50463,
  30155,
  12296,
  5243
)

Now we'll create a FIR filter that accepts a window function as the argument. This allows us to define new windows later on and retain the same FIR generator. It also allows us to independently size the FIR, knowing the window will be recalculated for different lengths or bitwidths. Since we are choosing the window at compile time, these coefficients are fixed.

In [20]:
// our FIR has parameterized window length, IO bitwidth, and windowing function
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy
  })

  // calculate the coefficients using the provided window function, mapping to UInts
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays
  // note: we avoid using a Vec here since we don't need dynamic indexing
  val delays = Seq.fill(length)(Wire(UInt(bitwidth.W))).scan(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults"
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs with bit growth
  val result = mults.reduce(_+&_)

  // connect output
  io.out := result
}

defined class MyFir

Those last three lines could be easily combined into one. Also notice how we've handled bitwidth growth conservatively to avoid loss.

<span style="color:blue">**Example: FIR Filter Tester**</span><br>
Let's test our FIR! Previously, we provided a custom golden model. This time we'll use Breeze, a Scala library of useful linear algebra and signal processing functions, as a golden model for our FIR filter. The code below compares the Chisel output with the golden model output, and any errors cause the tester to fail.

Try uncommenting the print statment at the end just after the expect call. Also try changing the window from triangular to Hamming.

In [21]:
// math imports
import scala.math.{pow, sin, Pi}
import breeze.signal.{filter, OptOverhang}
import breeze.signal.support.{CanFilter, FIRKernel1D}
import breeze.linalg.DenseVector

// test parameters
val length = 7
val bitwidth = 12 // must be less than 15, otherwise Int can't represent the data, need BigInt
val window = TriangularWindow

// test our FIR
Driver(() => new MyFir(length, bitwidth, window)) {
  c => new PeekPokeTester(c) {
    
    // test data
    val n = 100 // input length
    val sine_freq = 10
    val samp_freq = 100
    
    // sample data, scale to between 0 and 2^bitwidth
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => (max_value/2 + max_value/2*sin(2*Pi*sine_freq/samp_freq*i)).toInt)
    //println(s"input = ${sine.toArray.deep.mkString(", ")}")
    
    // coefficients
    val coeffs = window(length, bitwidth)
    //println(s"coeffs = ${coeffs.toArray.deep.mkString(", ")}")

    // use breeze filter as golden model; need to reverse coefficients
    val expected = filter(DenseVector(sine.toArray), 
                          FIRKernel1D(DenseVector(coeffs.reverse.toArray), 1.0, ""), 
                          OptOverhang.None)
    //println(s"exp_out = ${expected.toArray.deep.mkString(", ")}")

    // push data through our FIR and check the result
    reset(5)
    for (i <- 0 until n) {
      poke(c.io.in, sine(i))
      if (i >= length-1) { // wait for all registers to be initialized since we didn't zero-pad the data
        expect(c.io.out, expected(i-length+1))
        println(s"cycle $i, got ${peek(c.io.out)}, expect ${expected(i-length+1)}")
      }
      step(1)
    }
  }
}

[info] [0.002] Elaborating design...
[info] [1.074] Done elaborating.
Total FIRRTL Compile Time: 660.8 ms
Total FIRRTL Compile Time: 66.8 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1575039935808
[info] [0.605] cycle 6, got 43373193, expect 43373193
[info] [0.622] cycle 7, got 36413841, expect 36413841
[info] [0.637] cycle 8, got 25153536, expect 25153536
[info] [0.645] cycle 9, got 13893231, expect 13893231
[info] [0.669] cycle 10, got 6933879, expect 6933879
[info] [0.688] cycle 11, got 6933879, expect 6933879
[info] [0.698] cycle 12, got 13893231, expect 13893231
[info] [0.712] cycle 13, got 25153536, expect 25153536
[info] [0.713] cycle 14, got 36413841, expect 36413841
[info] [0.745] cycle 15, got 43373193, expect 43373193
[info] [0.747] cycle 16, got 43373193, expect 43373193
[info] [0.763] cycle 17, got 36413841, expect 36413841
[info] [0.779] cycle 18, got 25153536, expect 25153536
[info] [0.780] cycle 19, got 13893231, expect 13893231
[info] [0.788] cy

import scala.math.{pow, sin, Pi}

import breeze.signal.{filter, OptOverhang}

import breeze.signal.support.{CanFilter, FIRKernel1D}

import breeze.linalg.DenseVector

// test parameters

length: Int = 7
bitwidth: Int = 12
window: (Int, Int) => Seq[Int] = ammonite.$sess.cmd18$Helper$$Lambda$4216/0x0000000801ebe840@501cae04
res20_7: Boolean = true

---
# Chisel Exercises
Complete the following exercises to practice writing functions, using them as arguments to hardware generators, and avoiding mutable data.

<span style="color:red">**Exercise: Neural Network Neuron**</span><br>
Our first example will have you build a neuron, the building block of fully-connected layers in artificial neural networks. Neurons take inputs and a set of weights, one per input, and produce one output. The weights and inputs are multiplied and added, and the result is fed through an activation function. In this exercise, you will implement different activation functions and pass them as an argument to your neuron generator.

![Neuron](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png)

First, complete the following code to create a neuron generator. The parameter `inputs` gives the number of inputs. The parameter `act` is a function that implements the logic of the activation function. We'll make the inputs and outputs 16-bit fixed point values with 8 fractional bits.

In [22]:
class Neuron(inputs: Int, act: FixedPoint => FixedPoint) extends Module {
  val io = IO(new Bundle {
    val in      = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val weights = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val out     = Output(FixedPoint(16.W, 8.BP))
  })
  
    val netj = io.in.zip(io.weights).map{case (a,b)=>a*b}.reduce(_+_)
    io.out := act(netj)
    
}

defined class Neuron

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val mac = io.in.zip(io.weights).map{ case(a:FixedPoint, b:FixedPoint) => a*b}.reduce(_+_)
  io.out := act(mac)
</pre></article></div></section></div>

Now let's create some activation functions! We'll use a threshold of zero. Typical activation functions are the sigmoid function and the rectified linear unit (ReLU).

The sigmoid we'll use is called the [logistic function](https://en.wikipedia.org/wiki/Logistic_function), given by 

$logistic(x) = \cfrac{1}{1+e^{-\beta x}}$

where $\beta$ is the slope factor. However, computing the exponential function in hardware is quite challenging and expensive. We'll approximate this as the step function.

$step(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             1  & \text{if } x \gt 0
       \end{cases}$

The second function, the ReLU, is given by a similar formula.

$relu(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             x  & \text{if } x \gt 0
       \end{cases}$

Implement these two functions below. You can specify a fixed-point literal like `-3.14.F(8.BP)`. 

In [24]:
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)

Step: FixedPoint => FixedPoint = ammonite.$sess.cmd23$Helper$$Lambda$5728/0x00000008027c4040@21414e7e
ReLU: FixedPoint => FixedPoint = ammonite.$sess.cmd23$Helper$$Lambda$5729/0x00000008027c5040@5ad3c0f

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)
</pre></article></div></section></div>

Finally, let's create a tester that checks the correctness of our Neuron. With the step activation function, neurons may be used as logic gate approximators. Proper selection of weights and bias can perform binary functions. We'll test our neuron using AND logic. Complete the following tester to check our neuron with the step function.

Note that since the circuit is purely combinational, the `reset(5)` and `step(1)` calls are not necessary.

In [25]:
// test our Neuron 
Driver(() => new Neuron(2, Step)) {
  c => new PeekPokeTester(c) {
    
    val inputs = Seq(Seq(-1, -1), Seq(-1, 1), Seq(1, -1), Seq(1, 1))
    
    // make this a sequence of two values
    val weights = Seq(1,1)

    // push data through our Neuron and check the result (AND gate)
    reset(5)
    for (i <- inputs) {
      pokeFixedPoint(c.io.in(0), i(0))
      pokeFixedPoint(c.io.in(1), i(1))
      pokeFixedPoint(c.io.weights(0), weights(0))
      pokeFixedPoint(c.io.weights(1), weights(1))
      expectFixedPoint(c.io.out, if (i(0) + i(1) > 0) 1 else 0, "ERROR")
      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.111] Done elaborating.
Total FIRRTL Compile Time: 70.4 ms
Total FIRRTL Compile Time: 47.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575040410043
test Neuron Success: 0 tests passed in 14 cycles taking 0.052923 seconds
[info] [0.042] RAN 4 CYCLES PASSED


res24: Boolean = true

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val weights  = Seq(1.0, 1.0)
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)